In [1]:
from source.utils import *

In [2]:
template = {
  "accountId": "831650818513",
  "fragment": {
    "Parameters": {
      "GitHubUser": {
        "Type": "String",
        "Default": "ggiallo28"
      },
      "RepositoryName": {
        "Type": "String",
        "Default": "template-macro"
      },
      "BranchName": {
        "Type": "String",
        "Default": "master"
      },
      "GitHubToken": {
        "Type": "String",
        "Default": "3323ce281edb2c489c31aa7a2366b7d5045ac9aa"
      },
      "BucketName": {
        "Type": "String",
        "Default": "audioposts-site"
      },
      "TemplateKey": {
        "Type": "String",
        "Default": "test/sns-topics.yaml"
      }
    },
    "Resources": {
      "SNSTopicNested": {
        "Type": "Template::Git",
        "Properties": {
          "Mode": "Nested",
          "Provider": "GitHub",
          "Owner": {
            "Ref": "GitHubUser"
          },
          "Repo": {
            "Ref": "RepositoryName"
          },
          "Branch": {
            "Ref": "BranchName"
          },
          "OAuthToken": {
            "Ref": "GitHubToken"
          },
          "Path": {
            "Ref": "TemplateKey"
          },
          "Parameters": {
            "Name": "sns-topic-template-github-nested-stack"
          }
        }
      },
      "SNSTopiInline": {
        "Type": "Template::Git",
        "Properties": {
          "Mode": "Inline",
          "Repo": {
            "Ref": "RepositoryName"
          },
          "Branch": {
            "Ref": "BranchName"
          },
          "Provider": "Codecommit",
          "Parameters": {
            "Name": "sns-topic-template-codecommit-inline-stack"
          },
          "Path": {
            "Ref": "TemplateKey"
          }
        }
      },
      "SNSTopicS3": {
        "Type": "Template::S3",
        "Properties": {
          "Mode": "Inline",
          "Bucket": {
            "Ref": "BucketName"
          },
          "Key": {
            "Ref": "TemplateKey"
          },
          "Provider": "S3",
          "Parameters": {
            "Name": "sns-topic-template-s3-inline-stack"
          }
        }
      }
    }
  },
  "transformId": "831650818513::Template",
  "requestId": "zcca5cb0f0-78e6-4834-8198-398961421233",
  "region": "us-east-1",
  "params": {},
  "templateParameterValues": {
    "GitHubUser": "ggiallo28",
    "BucketName": "audioposts-site",
    "TemplateKey": "test/sns-topics.yaml",
    "RepositoryName": "template-macro",
    "BranchName": "master",
    "GitHubToken": "3323ce281edb2c489c31aa7a2366b7d5045ac9aa"
  }
}

In [3]:
tp_template = TemplateLoader.loads(template["fragment"])
print(tp_template.to_yaml())

Parameters:
  BranchName:
    Default: master
    Type: String
  BucketName:
    Default: audioposts-site
    Type: String
  GitHubToken:
    Default: 3323ce281edb2c489c31aa7a2366b7d5045ac9aa
    Type: String
  GitHubUser:
    Default: ggiallo28
    Type: String
  RepositoryName:
    Default: template-macro
    Type: String
  TemplateKey:
    Default: test/sns-topics.yaml
    Type: String
Resources:
  SNSTopiInline:
    Properties:
      Branch: !Ref 'BranchName'
      Mode: Inline
      Parameters:
        Name: sns-topic-template-codecommit-inline-stack
      Path: !Ref 'TemplateKey'
      Provider: Codecommit
      Repo: !Ref 'RepositoryName'
    Type: Template::Git
  SNSTopicNested:
    Properties:
      Branch: !Ref 'BranchName'
      Mode: Nested
      OAuthToken: !Ref 'GitHubToken'
      Owner: !Ref 'GitHubUser'
      Parameters:
        Name: sns-topic-template-github-nested-stack
      Path: !Ref 'TemplateKey'
      Provider: GitHub
      Repo: !Ref 'RepositoryName'
    Type: 

In [4]:
sns_template = {
    "AWSTemplateFormatVersion": "2010-09-09",
    "Description": "SNS Topics for alarms",
    "Parameters": {
        "SupportEmail": {
            "Type": "String",
            "Default": "support+18x7@xpeppers.com"
        },
        "AccountName": {
            "Type": "String",
            "Default": ""
        }
    },
    "Resources": {
        "UrgentPriorityAlarm": {
            "Type": "AWS::SNS::Topic",
            "Properties": {
                "DisplayName": {
                    "Fn::Sub": "${AccountName}-Urgent-Priority-Alarm"
                },
                "Subscription": [{
                    "Endpoint": {
                        "Ref": "SupportEmail"
                    },
                    "Protocol": "email"
                }],
                "TopicName": {
                    "Fn::Sub": "${AccountName}-Urgent-Priority-Alarm"
                }
            }
        },
        "HighPriorityAlarm": {
            "Type": "AWS::SNS::Topic",
            "Properties": {
                "DisplayName": {
                    "Fn::Sub": "${AccountName}-High-Priority-Alarm"
                },
                "Subscription": [{
                    "Endpoint": {
                        "Ref": "SupportEmail"
                    },
                    "Protocol": "email"
                }],
                "TopicName": {
                    "Fn::Sub": "${AccountName}-High-Priority-Alarm"
                }
            }
        },
        "MediumPriorityAlarm": {
            "Type": "AWS::SNS::Topic",
            "Properties": {
                "DisplayName": {
                    "Fn::Sub": "${AccountName}-Medium-Priority-Alarm"
                },
                "Subscription": [{
                    "Endpoint": {
                        "Ref": "SupportEmail"
                    },
                    "Protocol": "email"
                }],
                "TopicName": {
                    "Fn::Sub": "${AccountName}-Medium-Priority-Alarm"
                }
            }
        },
        "LowPriorityAlarm": {
            "Type": "AWS::SNS::Topic",
            "Properties": {
                "DisplayName": {
                    "Fn::Sub": "${AccountName}-Low-Priority-Alarm"
                },
                "Subscription": [{
                    "Endpoint": {
                        "Ref": "SupportEmail"
                    },
                    "Protocol": "email"
                }],
                "TopicName": {
                    "Fn::Sub": "${AccountName}-Low-Priority-Alarm"
                }
            }
        }
    },
    "Outputs": {
        "UrgentPriorityAlarm": {
            "Description": "Urgent priority SNS topic ARN",
            "Value": {
                "Ref": "UrgentPriorityAlarm"
            },
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-urgent"
                }
            }
        },
        "HighPriorityAlarm": {
            "Description": "High priority SNS topic ARN",
            "Value": {
                "Ref": "HighPriorityAlarm"
            },
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-high"
                }
            }
        },
        "MediumPriorityAlarm": {
            "Description": "Medium priority SNS topic ARN",
            "Value": {
                "Ref": "MediumPriorityAlarm"
            },
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-medium"
                }
            }
        },
        "LowPriorityAlarm": {
            "Description": "Low priority SNS topic ARN",
            "Value": {
                "Ref": "LowPriorityAlarm"
            },
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-low"
                }
            }
        }
    }
}

In [5]:
tempaltes = {}
for name in tp_template.resources:
    tempaltes[name] = TemplateLoader.loads(sns_template, prefix = name)

In [6]:
tempaltes[name].outputs['SNSTopicS3HighPriorityAlarm'].properties['Export'].data['Name']

{'Fn::Sub': 'SNSTopicS3-SNSTopiInline-SNSTopicNested-${AWS::StackName}-high'}

In [7]:
from troposphere import Template

empty_template = Template()
empty_template.description = "test"

empty_template.to_json()

'{\n    "Description": "test",\n    "Resources": {}\n}'

In [8]:
new_template = TemplateLoader.init()
    
for key in tempaltes:
    new_template += tempaltes[key]

In [9]:
print(new_template.to_yaml())

Description: This template is the result of the merge. [SNSTopicNested] SNS Topics
  for alarms [SNSTopiInline] SNS Topics for alarms [SNSTopicS3] SNS Topics for alarms
Outputs:
  SNSTopiInlineHighPriorityAlarm:
    Description: High priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-high'
    Value: !Ref 'SNSTopiInlineHighPriorityAlarm'
  SNSTopiInlineLowPriorityAlarm:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-low'
    Value: !Ref 'SNSTopiInlineLowPriorityAlarm'
  SNSTopiInlineMediumPriorityAlarm:
    Description: Medium priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-medium'
    Value: !Ref 'SNSTopiInlineMediumPriorityAlarm'
  SNSTopiInlineUrgentPriorityAlarm:
    Description: Urgent priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-urgent'
    Value: !Ref '

In [7]:
print(tempaltes['SNSTopicNested'].update('SNSTopicNested'))

{
    "AWSTemplateFormatVersion": "2010-09-09",
    "Description": "[SNSTopicNested] SNS Topics for alarms\n",
    "Outputs": {
        "SNSTopicNestedHighPriorityAlarm": {
            "Description": "High priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-high"
                }
            },
            "Value": {
                "Ref": "SNSTopicNestedHighPriorityAlarm"
            }
        },
        "SNSTopicNestedLowPriorityAlarm": {
            "Description": "Low priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-low"
                }
            },
            "Value": {
                "Ref": "SNSTopicNestedLowPriorityAlarm"
            }
        },
        "SNSTopicNestedMediumPriorityAlarm": {
            "Description": "Medium priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::

In [7]:
tempaltes['SNSTopicNested']['outputs']['SNSTopicNestedUrgentPriorityAlarm'].title
    
#attributes    
#    'properties',
# 'propnames',
# 'props',
# 'ref',
# 'resource',
# 'template',
# 'title',
# 'to_dict',
# 'validate',
# 'validate_title']

'SNSTopicNestedUrgentPriorityAlarm'

In [8]:
print(tempaltes['SNSTopicNested'].to_yaml())

AWSTemplateFormatVersion: '2010-09-09'
Description: "[SNSTopicNested] SNS Topics for alarms\n"
Outputs:
  SNSTopicNestedHighPriorityAlarm:
    Description: High priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-high'
    Value: !Ref 'HighPriorityAlarm'
  SNSTopicNestedLowPriorityAlarm:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-low'
    Value: !Ref 'LowPriorityAlarm'
  SNSTopicNestedMediumPriorityAlarm:
    Description: Medium priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-medium'
    Value: !Ref 'MediumPriorityAlarm'
  SNSTopicNestedUrgentPriorityAlarm:
    Description: Urgent priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-urgent'
    Value: !Ref 'UrgentPriorityAlarm'
Parameters:
  SNSTopicNestedAccountName:
    Default: ''
    Type: String
  SNSTopicNestedSupportEmail:
    Default: support+18x7@xpeppers.com
    Type: String
Resources:
  SNSTopicNestedHighPriorityAlarm

In [22]:
tempaltes['SNSTopicNested'].outputs['SNSTopicNestedHighPriorityAlarm'].title

'HighPriorityAlarm'

In [17]:
dir(tempaltes['SNSTopicNested'].outputs['SNSTopicNestedHighPriorityAlarm'])

['Ref',
 '_BaseAWSObject__initialized',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_from_dict',
 '_raise_type',
 '_validate_props',
 'add_to_template',
 'attributes',
 'do_validation',
 'from_dict',
 'no_validation',
 'properties',
 'propnames',
 'props',
 'ref',
 'resource',
 'template',
 'title',
 'to_dict',
 'validate',
 'validate_title']

In [133]:
new_template.parameters = {
    **tp_sns_template.parameters, 
    **tp_sns_template.parameters, 
    **tp_sns_template.parameters
}

new_template.conditions = {
    **tp_sns_template.conditions, 
    **tp_sns_template.conditions, 
    **tp_sns_template.conditions
}

new_template.description = 'test'
#    **tp_sns_template.description, 
#    **tp_sns_template.description, 
#    **tp_sns_template.description

new_template.mappings = {
    **tp_sns_template.mappings, 
    **tp_sns_template.mappings, 
    **tp_sns_template.mappings
}

new_template.metadata = {
    **tp_sns_template.metadata, 
    **tp_sns_template.metadata, 
    **tp_sns_template.metadata
}

new_template.outputs = {
    **tp_sns_template.outputs, 
    **tp_sns_template.outputs, 
    **tp_sns_template.outputs
}

new_template.props = {
    **tp_sns_template.props, 
    **tp_sns_template.props, 
    **tp_sns_template.props
}

new_template.resources = {
    **tp_sns_template.resources, 
    **tp_sns_template.resources, 
    **tp_sns_template.resources
}

new_template.rules = {
    **tp_sns_template.rules, 
    **tp_sns_template.rules, 
    **tp_sns_template.rules
}

transform = [tp_sns_template.transform, tp_sns_template.transform, tp_sns_template.transform]
new_template.transform = transform

version = [tp_sns_template.version, tp_sns_template.version, tp_sns_template.version]
new_template.version = version

In [137]:
new_template.resources

{'UrgentPriorityAlarm': <troposphere.sns.Topic at 0x10e80f0d0>,
 'HighPriorityAlarm': <troposphere.sns.Topic at 0x10e86a1d0>,
 'MediumPriorityAlarm': <troposphere.sns.Topic at 0x10e86a4d0>,
 'LowPriorityAlarm': <troposphere.sns.Topic at 0x10e86a6d0>}